In [7]:
import pandas as pd
import json

import semantic_kernel as sk
from semantic_kernel.core_plugins import ConversationSummaryPlugin, TextPlugin
from semantic_kernel.text import (
    split_markdown_lines,
    split_markdown_paragraph,
    split_plaintext_lines,
    split_plaintext_paragraph,
)


In [28]:
enc_notes = pd.read_csv('../src/Api/data/notes_no_PHI.csv')

print(enc_notes['NOTE_TEXT'][0])

Depression Screen 
Positive:  PHQ-2 score >= 3 or PHQ-9 score >= 9
PHQ-2 Total: 0 (4/15/2024 10:20 AM)
 
DEPRESSION PLAN OF CARE
Her depression screen was negative.
 
 
Sky Ryder is a 27 y.o. female

Chief Complaint
Patient presents with
•	Establish Care
•	Physical
 	 	With annual labs 
 
 
Pt is here to establish care
Reviewed previous PCP notes and labs
 
Pt states she's been having chronic pain of right knee for many years. She had surgery in 2016 and continues to have pain. She would like referral to ortho
 
Doing well on celexa daily
 
She is currently on semaglutide through weight loss clinic and doing well. 
 
Previous labs noted low HDL and elevated CRP
 
Thyroid Disorder - was getting medication through IHC but states they never monitored lab
 She is seen today for follow up of hypothyroidism. Current symptoms include: denies fatigue, weight changes, heat/cold intolerance, bowel/skin changes or CVS symptoms. Denies swallowing difficulty, voice change or masses noted in the nec

## Extract sections from Note
1. HPI (startswith 'Chief Complaint')
1. Lab Results
1. Assessment and Plan

In [19]:
def split_text(text, max_tokens=25):
    return split_plaintext_paragraph(text, max_tokens=max_tokens, token_counter=lambda x: len(x))

In [ ]:
text = ['Bacon ipsum dolor amet turkey short ribs tenderloin ball tip bresaola shankle porchetta. Chicken ground round salami venison sausage. Drumstick porchetta hamburger flank pork loin. Prosciutto buffalo pancetta andouille, tail brisket ball tip pork belly beef. Kevin beef ribs chuck brisket. Pancetta tail swine chislic shoulder shankle pastrami turkey sirloin bacon hamburger pig. Porchetta shoulder flank meatloaf venison.']

result = split_plaintext_paragraph(text, max_tokens=25, token_counter=lambda x: len(x))

result

In [21]:
enc_notes['summary'] = enc_notes['NOTE_TEXT'].apply(lambda x: split_text([x]))

# for item in enc_notes, print the summary
for index, row in enc_notes.iterrows():
    print(row['summary'])


['Depression Screen', 'Positive:', 'PHQ-2 score >= 3', 'or PHQ-9 score >= 9', 'PHQ-2 Total:', '0 (4/15/2024 10:20 AM)', 'DEPRESSION PLAN OF CARE', 'Her depression', 'screen was negative.', 'Sky Ryder is a 27 y.', 'o. female', 'Chief Complaint', 'Patient presents with', '•\tEstablish Care', '•\tPhysical', 'With annual labs', 'Pt is here to', 'establish care', 'Reviewed previous', 'PCP notes and labs', "Pt states she's", 'been having chronic', 'pain of right knee', 'for many years.', 'She had surgery in 2016', 'and continues', 'to have pain.', 'She would like', 'referral to ortho', 'Doing well on', 'celexa daily', 'She is currently on', 'semaglutide through', 'weight loss clinic', 'and doing well.', 'Previous labs noted', 'low HDL and elevated CRP', 'Thyroid Disorder -', 'was getting medication', 'through IHC but states', 'they never monitored lab', 'She is seen today for', 'follow up of', 'hypothyroidism.', 'Current symptoms include:', 'denies fatigue,', 'weight changes,', 'heat/cold in

## Get lab components from lab order

In [29]:
lab_results = pd.read_csv('../src/Api/data/lab_rslt_component_no_PHI.csv')

# filter lab results by ORD_ID == 279149741
lab_results = lab_results.loc[lab_results['ORD_ID'] == 279149741]
cols = ['CMPNT_NM', 'RSLT_VALUE_TXT', 'REFER_UNIT', 'REFER_VALUE']
lab_results = lab_results[cols]

lab_results
result = lab_results.to_json(orient="records")

# Summarize the lab result values
print(result)

[{"CMPNT_NM":"HDL","RSLT_VALUE_TXT":111.0,"REFER_UNIT":"mg\/dL","REFER_VALUE":"> OR = 50"},{"CMPNT_NM":"CHOLESTEROL","RSLT_VALUE_TXT":225.0,"REFER_UNIT":"mg\/dL","REFER_VALUE":"<200"},{"CMPNT_NM":"TRIGLYCERIDE","RSLT_VALUE_TXT":89.0,"REFER_UNIT":"mg\/dL","REFER_VALUE":"<150"},{"CMPNT_NM":"LDL CALCULATED","RSLT_VALUE_TXT":96.0,"REFER_UNIT":"mg\/dL (calc)","REFER_VALUE":null},{"CMPNT_NM":"CHOL\/HDL RATIO","RSLT_VALUE_TXT":2.0,"REFER_UNIT":"(calc)","REFER_VALUE":"<5.0"},{"CMPNT_NM":"TOTAL NON-HDL CHOLESTEROL (LDL+VLDL)","RSLT_VALUE_TXT":114.0,"REFER_UNIT":"mg\/dL (calc)","REFER_VALUE":"<130"}]
